# Example for Horiba FHR

This driver has been tested with a FHR1000 spectrometer with one dual grating turret, an entrance slit, exit slit, and an exit slit selection mirror.

The dll unfortunately does not have any version information. It was shipped with LabSpec 6.5.1. The supporting information is contained in a word document called `SpeControl_dll.docx`.

In [1]:
import pathlib
import tempfile
from qcodes_contrib_drivers.drivers.Horiba.Horiba_FHR import HoribaFHR

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Flash\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Flash\.qcodes\logs\231006-740-qcodes.log


In [2]:
dll_dir = r'C:\HORIBA\Software\SDK FHR Express'
config_file = pathlib.Path(tempfile.mkdtemp(), 'config.ini')
config_file.write_text(
"""[Firmware]
VersionNumber=123456
SerialNumber=1234567890
==================================================
[Port]
ComPort=13
Baudrate=115200
Timeout=60000
==================================================
[Spectrometer]
Focal=1000
CoefficientOfAngle=0.20071286
Board=1
SpeedMin=500
SpeedMax=280000
Acceleration=400
Backlash=-12500
MotorStepUnit=2
Reverse=0
GratingNumber=2
SlitNumber=2
==================================================
[Grating1]
Name=1800 gr/mm
Value=1800
AddrAxe=5
Offset=54940
Shift=0
MinNm=50
MaxNm=750
CoefficientOfLinearity=1.000069196
==================================================
[Grating2]
Name=600 gr/mm
Value=600
AddrAxe=9
Offset=2554965
Shift=0
MinNm=50
MaxNm=2000
CoefficientOfLinearity=0.999935906
==================================================
[Slit1]
Name=Front entrance slit
AddrAxe=1
Offset=450
Minum=0
Maxum=2000
Coeffum=1
SpeedMin=50
SpeedMax=500
Acceleration=150
Backlash=100
MotorStepUnit=1
Reverse=1
==================================================
[Slit2]
Name=Side exit slit
AddrAxe=4
Offset=450
Minum=0
Maxum=2000
Coeffum=1
SpeedMin=50
SpeedMax=500
Acceleration=150
Backlash=100
MotorStepUnit=1
Reverse=1
==================================================
[Mirror2]
Name=Exit mirror
AddrAxe=7
Delayms=100
DutyCycle%=30
==================================================
""")

1326

In [3]:
spe = HoribaFHR('spe', dll_dir, config_file,
                dc_val_mappings={2: {'front': 0, 'side': 1}})

Connected to: Horiba FHR1000 (serial:1234567890, firmware:123456) in 1.48s


The `dc_val_mappings` parameter allows to map the binary positions of the DC motors to a more inuititive string.

The various controllers are stored as `ChannelList`s:

In [4]:
grating_1800, grating_600 = spe.gratings
slit_entrance, slit_exit = spe.slits
mirror, = spe.mirrors

The main parameter each channel has is `position`. For `SlitChannel`s, there is also `width`, which should be preferred since it represents the actual opening width of the slit instead of the position of the stepper motor. To initialize the motors, meaning

- move to zero order for gratings
- completely close for slits

call `init()`. Setting the position of a grating will turn the turret if the other grating was selected before:

In [ ]:
grating_1800.position(600e3)  # pm!
grating_600.position(600e3)  # will turn the turret by 180 degrees + X.

In [5]:
spe.close()